In [1]:
!pip install datasets
import pandas as pd
#import datasets as ds
from datasets import Dataset, DatasetDict
import EDAHelper as hlpr
#from datasets import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k45li\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\k45li\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k45li\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('resume.csv')
df['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [3]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ADVOCATE                  118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [4]:
df.columns

Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')

In [5]:
# Remove unnecessary columns 
columns_to_remove = ['ID', 'Resume_html']
df = df.drop(columns=columns_to_remove)

In [6]:
# Rename Resume_str to 'Resume
df.rename(columns={'Resume_str': 'Resume'}, inplace=True)

In [7]:
# Minor cleanup to clear all leading and trailing spaces
df['Resume']=df['Resume'].str.strip()

In [8]:
hlpr.show_blank_cells(df)

Row positions containing blank cells or cells with an empty string:
[656]


In [9]:
df.shape

(2484, 2)

In [10]:
df.drop(656, inplace=True)

In [11]:
df.shape

(2483, 2)

In [12]:
df.info

<bound method DataFrame.info of                                                  Resume  Category
0     HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADM...        HR
1     HR SPECIALIST, US HR OPERATIONS       Summary ...        HR
2     HR DIRECTOR       Summary      Over 20 years e...        HR
3     HR SPECIALIST       Summary    Dedicated, Driv...        HR
4     HR MANAGER         Skill Highlights           ...        HR
...                                                 ...       ...
2479  RANK: SGT/E-5 NON- COMMISSIONED OFFICER IN CHA...  AVIATION
2480  GOVERNMENT RELATIONS, COMMUNICATIONS AND ORGAN...  AVIATION
2481  GEEK SQUAD AGENT         Professional Profile ...  AVIATION
2482  PROGRAM DIRECTOR / OFFICE MANAGER           Su...  AVIATION
2483  STOREKEEPER II       Professional Summary    T...  AVIATION

[2483 rows x 2 columns]>

In [13]:
df.to_csv('resume_classifier.csv')  # for my further anolysis

df=pd.read_csv('resume_classifier.csv', sep=',', index_col = 0)

In [14]:
df

,Resume,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADM...,HR
1,"HR SPECIALIST, US HR OPERATIONS Summary ...",HR
2,HR DIRECTOR Summary Over 20 years e...,HR
3,"HR SPECIALIST Summary Dedicated, Driv...",HR
4,HR MANAGER Skill Highlights ...,HR
...,...,...
2479,RANK: SGT/E-5 NON- COMMISSIONED OFFICER IN CHA...,AVIATION
2480,"GOVERNMENT RELATIONS, COMMUNICATIONS AND ORGAN...",AVIATION
2481,GEEK SQUAD AGENT Professional Profile ...,AVIATION
2482,PROGRAM DIRECTOR / OFFICE MANAGER Su...,AVIATION


In [15]:
# Load the dataframe in Hugging Face format
dataset = Dataset.from_pandas(df)

In [16]:
dataset

Dataset({
    features: ['Resume', 'Category', '__index_level_0__'],
    num_rows: 2483
})

In [17]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame
# Splitting into 70% train, 30% remaining
train_df, remaining_df = train_test_split(df, test_size=0.3, random_state=42)

# Splitting the remaining 30% into 20% and 10%
test_df, validation_df = train_test_split(remaining_df, test_size=0.33, random_state=42)

# Displaying sizes of each split
print("Train DataFrame size:", len(train_df))
print("Test DataFrame size:", len(test_df))
print("Validation DataFrame size:", len(validation_df))

Train DataFrame size: 1738
Test DataFrame size: 499
Validation DataFrame size: 246


In [18]:
train_s=Dataset.from_pandas(train_df)
test_s=Dataset.from_pandas(test_df)
validation_s=Dataset.from_pandas(validation_df)

In [19]:
train_s

Dataset({
    features: ['Resume', 'Category', '__index_level_0__'],
    num_rows: 1738
})

In [20]:
validation_s

Dataset({
    features: ['Resume', 'Category', '__index_level_0__'],
    num_rows: 246
})

In [21]:
train_s = train_s.remove_columns('__index_level_0__')
test_s = test_s.remove_columns('__index_level_0__')
validation_s = validation_s.remove_columns('__index_level_0__')

In [22]:
test_s

Dataset({
    features: ['Resume', 'Category'],
    num_rows: 499
})

In [23]:
ds=DatasetDict()

In [24]:
ds['train']=train_s
ds['validation']=validation_s
ds['test']=test_s

In [25]:
ds

DatasetDict({
    train: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 1738
    })
    validation: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 246
    })
    test: Dataset({
        features: ['Resume', 'Category'],
        num_rows: 499
    })
})

In [26]:
ds['test'][1]

{'Resume': "SERGEANT E-5 AVIATION SUPPLY SPECIALIST       Summary    A highly motivated and ambitious individual\nable to give timely and accurate advice, guidance and support to team members\nand individuals. Possess excellent management skills and have the ability to\nwork with the minimum of supervision whilst supporting fellow team members.\nHave a proven ability to lead by example, consistently hit targets, improve\nbest practices and organize time efficiently. Now looking forward to make a\nfurther contribution as part of your team and am hopeful for the genuine\nopportunity for progression.      Highlights          Inventory tracking  Budget management  Shipment consolidations planning  Decisive  Inventory control  Warehouse and fulfillment      Results-oriented  Logistics network analysis  Detail-oriented  Strategic planner  Cost reduction  Flexible  Works well independently  Customer service-oriented            Accomplishments     ·\xa0\xa0\nReceived\na Navy and Marine Corps A

In [27]:
from huggingface_hub import notebook_login
notebook_login()

In [28]:
ds.push_to_hub("resume_classifierV1")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/K45lim/resume_classifierV1/commit/65294415a460a49653b23739622bb87b5221c74e', commit_message='Upload dataset', commit_description='', oid='65294415a460a49653b23739622bb87b5221c74e', pr_url=None, pr_revision=None, pr_num=None)